# INSTRUÇÃO PRÁTICA - IP-P008
## Revisao e consolidaçao do conteúdo de __Pandas__ avançado.
Dados de estações metereologicas de Salvador dos anos de 2003, 2013 e 2023, diponiveis em: [Instituto Nacional de Meteorologia](https://portal.inmet.gov.br/dadoshistoricos). 

### Exercicio 1: Tratando series temporais por tempos
1. Carregando dados utilizando `DatetimeIndex`

In [35]:
import pandas as pd
import numpy as np

estacao_ssa_2003 = pd.read_pickle('datasets/tratados/estacao_Salvador_2003.pkl')
estacao_ssa_2013 = pd.read_pickle('datasets/tratados/estacao_Salvador_2013.pkl')
estacao_ssa_2023 = pd.read_pickle('datasets/tratados/estacao_Salvador_2023.pkl')

In [36]:
# Cria o DateTimeIndex a partir da coluna DATA (YYYY-MM-DD) e HORA (UTC)
estacao_ssa_2003_index = pd.to_datetime(estacao_ssa_2003['DATA (YYYY-MM-DD)'] + ' ' + estacao_ssa_2003['HORA (UTC)'], utc=True)
estacao_ssa_2013_index = pd.to_datetime(estacao_ssa_2013['DATA (YYYY-MM-DD)'] + ' ' + estacao_ssa_2013['HORA (UTC)'], utc=True)
estacao_ssa_2023_index = pd.to_datetime(estacao_ssa_2023['Data'] + ' ' + estacao_ssa_2023['Hora UTC'], utc=True)

# Remova as colunas originais DATA (YYYY-MM-DD) e HORA (UTC)
estacao_ssa_2003.drop(['DATA (YYYY-MM-DD)', 'HORA (UTC)'], axis=1, inplace=True)
estacao_ssa_2013.drop(['DATA (YYYY-MM-DD)', 'HORA (UTC)'], axis=1, inplace=True)
estacao_ssa_2023.drop(['Data', 'Hora UTC'], axis=1, inplace=True)

# Define a coluna DATA HORA como índice
estacao_ssa_2003.index = estacao_ssa_2003_index
estacao_ssa_2013.index = estacao_ssa_2013_index
estacao_ssa_2023.index = estacao_ssa_2023_index

2. Juntando os dois datasets em um só

In [37]:
estacao_ssa = pd.concat([estacao_ssa_2003, estacao_ssa_2013, estacao_ssa_2023])

3. Analisando os dataframes

##### Média, Mínimo e Máximo

In [38]:
# Média
estacao_ssa.groupby(estacao_ssa.index.year).mean()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
2003,0.254141,25.631222,21.428161,26.073715,25.202994,21.868940,21.013844
2013,0.229320,25.929212,21.862606,26.472570,NaN,22.340487,21.416875
2023,0.174783,26.035347,21.875697,26.534308,25.558257,22.333440,21.420101


In [39]:
# Mínimo
estacao_ssa.groupby(estacao_ssa.index.year).min()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
2003,0.0,20.3,14.7,20.7,20.2,15.3,13.6
2013,0.0,19.5,17.4,20.1,NaN,18.0,-1.8
2023,0.0,20.5,15.2,20.7,20.5,16.2,14.8


In [40]:
# Máximo
estacao_ssa.groupby(estacao_ssa.index.year).max()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
2003,52.0,39.7,26.6,43.3,33.0,26.7,24.7
2013,52.0,34.1,26.0,34.6,NaN,26.8,24.5
2023,38.4,33.8,26.2,34.1,32.4,27.4,25.1


##### Mês de Janeiro de cada Ano

In [41]:
# Média
dados_janeiro = estacao_ssa[estacao_ssa.index.month == 1]

dados_janeiro.groupby(dados_janeiro.index.year).mean()


,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
2003,0.000660,27.419472,22.444884,27.891333,26.968333,22.747333,22.146333
2013,0.050000,27.310215,22.230376,27.882661,NaN,22.676747,21.788441
2023,0.167742,26.763038,22.569892,27.281855,26.267608,23.051747,22.094355


In [42]:
# Mínimo
dados_janeiro.groupby(dados_janeiro.index.year).min()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
2003,0.0,23.7,20.3,23.8,23.7,21.5,19.7
2013,0.0,23.2,18.2,23.4,NaN,20.0,17.6
2023,0.0,22.3,20.0,22.7,22.1,20.7,19.1


In [43]:
# Máximo
dados_janeiro.groupby(dados_janeiro.index.year).max()

,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)
2003,0.2,32.0,23.7,32.9,31.6,24.1,23.6
2013,5.8,34.1,25.3,34.6,NaN,25.5,24.3
2023,31.6,31.5,26.2,31.7,30.7,27.0,24.3


##### Estações

In [44]:
verao = estacao_ssa[(estacao_ssa.index.month == 1) | (estacao_ssa.index.month == 2) | (estacao_ssa.index.month == 3)]
outono = estacao_ssa[(estacao_ssa.index.month == 4) | (estacao_ssa.index.month == 5) | (estacao_ssa.index.month == 6)]
inverno = estacao_ssa[(estacao_ssa.index.month == 7) | (estacao_ssa.index.month == 8) | (estacao_ssa.index.month == 9)]
primavera = estacao_ssa[(estacao_ssa.index.month == 10) | (estacao_ssa.index.month == 11) | (estacao_ssa.index.month == 12)]

In [45]:
# Média

print(verao.groupby(verao.index.year).mean())
print(outono.groupby(outono.index.year).mean())
print(inverno.groupby(inverno.index.year).mean())
print(primavera.groupby(primavera.index.year).mean())

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                          0.188392   
2013                          0.049743   
2023                          0.173889   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                     27.089593   
2013                                     27.611734   
2023                                     26.926759   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                             22.592929   
2013                             22.275129   
2023                             22.719491   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                   27.559358   
2013                                   28.209911   
2023                                   27.457315   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                   26.644051   
2013                                         NaN   
2023                                   26.418148   

      TEMPE

In [46]:
# Mínimo

print(verao.groupby(verao.index.year).min())
print(outono.groupby(outono.index.year).min())
print(inverno.groupby(inverno.index.year).min())
print(primavera.groupby(primavera.index.year).min())

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                               0.0   
2013                               0.0   
2023                               0.0   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                          21.8   
2013                                          23.1   
2023                                          22.3   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                                  19.0   
2013                                  18.2   
2023                                  19.4   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                        22.1   
2013                                        23.3   
2023                                        22.7   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                        20.9   
2013                                         NaN   
2023                                        22.1   

      TEMPE

In [47]:
# Máximo

print(verao.groupby(verao.index.year).max())
print(outono.groupby(outono.index.year).max())
print(inverno.groupby(inverno.index.year).max())
print(primavera.groupby(primavera.index.year).max())

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                              21.8   
2013                               9.8   
2023                              31.6   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                          32.0   
2013                                          34.1   
2023                                          32.2   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                                  25.3   
2013                                  26.0   
2023                                  26.2   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                        32.9   
2013                                        34.6   
2023                                        32.8   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                        31.6   
2013                                         NaN   
2023                                        31.6   

      TEMPE

##### Intervalo de tempo (Madrugada, Manha, Tarde e Noite)

In [48]:
dados_dia_28_set = estacao_ssa[(estacao_ssa.index.month == 9) & (estacao_ssa.index.day == 28)]



In [49]:
# Média Madrugada

media_madrugada = dados_dia_28_set[(dados_dia_28_set.index.hour >= 0) & (dados_dia_28_set.index.hour < 6)]
media_madrugada_por_ano = media_madrugada.groupby(media_madrugada.index.year).mean()

print(media_madrugada_por_ano)

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                               0.0   
2013                               0.0   
2023                               0.0   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                     23.983333   
2013                                     23.516667   
2023                                     24.683333   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                             21.666667   
2013                             20.066667   
2023                             20.933333   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                   24.233333   
2013                                   23.716667   
2023                                   25.000000   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                   23.966667   
2013                                         NaN   
2023                                   24.650000   

      TEMPE

In [50]:
# Média Manhã

media_manha = dados_dia_28_set[(dados_dia_28_set.index.hour >= 6) & (dados_dia_28_set.index.hour < 12)]
media_manha_por_ano = media_manha.groupby(media_manha.index.year).mean()

print(media_manha_por_ano)

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                               0.0   
2013                               0.0   
2023                               0.0   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                     23.833333   
2013                                     24.216667   
2023                                     23.933333   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                             21.500000   
2013                             20.666667   
2023                             21.166667   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                   23.983333   
2013                                   24.366667   
2023                                   24.166667   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                   23.283333   
2013                                         NaN   
2023                                   23.316667   

      TEMPE

In [51]:
# Média Tarde

media_tarde = dados_dia_28_set[(dados_dia_28_set.index.hour >= 12) & (dados_dia_28_set.index.hour < 18)]
media_tarde_por_ano = media_tarde.groupby(media_tarde.index.year).mean()

print(media_tarde_por_ano)

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                               0.0   
2013                               0.0   
2023                               0.0   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                     28.416667   
2013                                     27.150000   
2023                                     30.100000   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                             21.966667   
2013                             21.700000   
2023                             21.050000   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                       28.85   
2013                                       28.05   
2023                                       30.55   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                   27.266667   
2013                                         NaN   
2023                                   28.783333   

      TEMPE

In [52]:
# Média Noite

media_noite = dados_dia_28_set[(dados_dia_28_set.index.hour >= 18) & (dados_dia_28_set.index.hour < 24)]
media_noite_por_ano = media_noite.groupby(media_noite.index.year).mean()

print(media_noite_por_ano)

      PRECIPITAÇÃO TOTAL, HORÁRIO (mm)  \
2003                               0.0   
2013                               0.0   
2023                               0.0   

      TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)  \
2003                                     25.816667   
2013                                     24.516667   
2023                                     26.733333   

      TEMPERATURA DO PONTO DE ORVALHO (°C)  \
2003                             21.833333   
2013                             21.533333   
2023                             21.333333   

      TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)  \
2003                                   26.483333   
2013                                   25.200000   
2023                                   27.733333   

      TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)  \
2003                                   25.733333   
2013                                         NaN   
2023                                   26.716667   

      TEMPE

### Exercicio 2: Utilizando eval() e query()
1. Comparação de desempenho